In [70]:
# IR PS3

In [119]:
f_time_all = open("Z:/20XW86-IR-LAB/ws3/dataset-info/TIME.ALL", 'r')

In [120]:
def parse_documents(f):
    documents = []
    fstr = f.read()
    regex = r"\*TEXT \d{3} \d{2}/\d{2}/\d{2} PAGE \d{3}\n\n"
    
    iters = []
    result = re.finditer(regex, fstr, re.DOTALL)
    for key in result:
        iters.append(key)
    
    for i in range(len(iters)-1):
        i1 = iters[i].span()[1]
        i2 = iters[i+1].span()[0]
        
        documents.append(fstr[i1:i2])
    
    return documents

In [121]:
docs = parse_documents(f_time_all)